In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'SVHN.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  #train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  #print('Training set', train_dataset.shape, train_labels.shape)
  #print('Validation set', valid_dataset.shape, valid_labels.shape)
  #print('Test set', test_dataset.shape, test_labels.shape)

In [6]:
pickle_file = 'SVHN1.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset1 = save['train_dataset1']
  del save  # hint to help gc free up memory

pickle_file = 'SVHN2.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset2 = save['train_dataset2']
  del save  # hint to help gc free up memory

pickle_file = 'SVHN3.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset3 = save['train_dataset3']
  del save  # hint to help gc free up memory

In [7]:
train_dataset = np.concatenate((train_dataset1, train_dataset2, train_dataset3), axis=0)
del train_dataset1, train_dataset2, train_dataset3
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (598388, 32, 32) (598388,)
Validation set (6000, 32, 32) (6000,)
Test set (26032, 32, 32) (26032,)


In [8]:
image_size = 32
num_labels = 10
num_channels = 1 # grayscale

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = labels.astype(np.int32)
  return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (598388, 32, 32, 1) (598388,)
Validation set (6000, 32, 32, 1) (6000,)
Test set (26032, 32, 32, 1) (26032,)


In [9]:
def LecunLCN(X, image_shape, threshold=1e-4, radius=7, use_divisor=True):
    """Local Contrast Normalization"""
    """[http://yann.lecun.com/exdb/publis/pdf/jarrett-iccv-09.pdf]"""

    # Get Gaussian filter
    filter_shape = (radius, radius, image_shape[3], 1)

    #self.filters = theano.shared(self.gaussian_filter(filter_shape), borrow=True)
    filters = gaussian_filter(filter_shape)
    X = tf.convert_to_tensor(X, dtype=tf.float32)
    # Compute the Guassian weighted average by means of convolution
    convout = tf.nn.conv2d(X, filters, [1,1,1,1], 'SAME')

    # Subtractive step
    mid = int(np.floor(filter_shape[1] / 2.))

    # Make filter dimension broadcastable and subtract
    centered_X = tf.sub(X, convout)

    # Boolean marks whether or not to perform divisive step
    if use_divisor:
        # Note that the local variances can be computed by using the centered_X
        # tensor. If we convolve this with the mean filter, that should give us
        # the variance at each point. We simply take the square root to get our
        # denominator

        # Compute variances
        sum_sqr_XX = tf.nn.conv2d(tf.square(centered_X), filters, [1,1,1,1], 'SAME')

        # Take square root to get local standard deviation
        denom = tf.sqrt(sum_sqr_XX)

        per_img_mean = tf.reduce_mean(denom)
        divisor = tf.maximum(per_img_mean, denom)
        # Divisise step
        new_X = tf.truediv(centered_X, tf.maximum(divisor, threshold))
    else:
        new_X = centered_X

    return new_X


def gaussian_filter(kernel_shape):
    x = np.zeros(kernel_shape, dtype = float)
    mid = np.floor(kernel_shape[0] / 2.)
    
    for kernel_idx in xrange(0, kernel_shape[2]):
        for i in xrange(0, kernel_shape[0]):
            for j in xrange(0, kernel_shape[1]):
                x[i, j, kernel_idx, 0] = gauss(i - mid, j - mid)
    
    return tf.convert_to_tensor(x / np.sum(x), dtype=tf.float32)

def gauss(x, y, sigma=3.0):
    Z = 2 * np.pi * sigma ** 2
    return  1. / Z * np.exp(-(x ** 2 + y ** 2) / (2. * sigma ** 2))

In [10]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == labels)
          / predictions.shape[0])

In [26]:
batch_size = 64
patch_size = 5
depth1 = 16
depth2 = 32
depth3 = 64
num_hidden1 = 64
num_hidden2 = 16
shape=[batch_size, image_size, image_size, num_channels]

# Construct a 7-layer CNN.
# C1: convolutional layer, batch_size x 28 x 28 x 16, convolution size: 5 x 5 x 1 x 16
# S2: sub-sampling layer, batch_size x 14 x 14 x 16
# C3: convolutional layer, batch_size x 10 x 10 x 32, convolution size: 5 x 5 x 16 x 32
# S4: sub-sampling layer, batch_size x 5 x 5 x 32
# C5: convolutional layer, batch_size x 1 x 1 x 64, convolution size: 5 x 5 x 32 x 64
# Dropout
# F6: fully-connected layer, weight size: 64 x 16
# Output layer, weight size: 16 x 10

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.int64, shape=(batch_size))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.get_variable("W1", shape=[patch_size, patch_size, num_channels, depth1],\
           initializer=tf.contrib.layers.xavier_initializer_conv2d())
  layer1_biases = tf.Variable(tf.constant(1.0, shape=[depth1]), name='B1')
  layer2_weights = tf.get_variable("W2", shape=[patch_size, patch_size, depth1, depth2],\
           initializer=tf.contrib.layers.xavier_initializer_conv2d())
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth2]), name='B2')
  layer3_weights = tf.get_variable("W3", shape=[patch_size, patch_size, depth2, num_hidden1],\
           initializer=tf.contrib.layers.xavier_initializer_conv2d())
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden1]), name='B3')
  layer4_weights = tf.get_variable("W4", shape=[num_hidden1, num_hidden2],\
           initializer=tf.contrib.layers.xavier_initializer())
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden2]), name='B4')
  layer5_weights = tf.get_variable("W5", shape=[num_hidden2, num_labels],\
           initializer=tf.contrib.layers.xavier_initializer())
  layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='B5')
  
  # Model.
  def model(data, keep_prob, shape):
    LCN = LecunLCN(data, shape)
    conv = tf.nn.conv2d(LCN, layer1_weights, [1,1,1,1], 'VALID', name='C1')
    hidden = tf.nn.relu(conv + layer1_biases)
    lrn = tf.nn.local_response_normalization(hidden)
    sub = tf.nn.max_pool(lrn, [1,2,2,1], [1,2,2,1], 'SAME', name='S2')
    conv = tf.nn.conv2d(sub, layer2_weights, [1,1,1,1], padding='VALID', name='C3')
    hidden = tf.nn.relu(conv + layer2_biases)
    lrn = tf.nn.local_response_normalization(hidden)
    sub = tf.nn.max_pool(lrn, [1,2,2,1], [1,2,2,1], 'SAME', name='S4')
    conv = tf.nn.conv2d(sub, layer3_weights, [1,1,1,1], padding='VALID', name='C5')
    hidden = tf.nn.relu(conv + layer3_biases)
    hidden = tf.nn.dropout(hidden, keep_prob)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer4_weights) + layer4_biases)
    return tf.matmul(hidden, layer5_weights) + layer5_biases
  
  # Training computation.
  logits = model(tf_train_dataset, 0.9375, shape)
  loss = tf.reduce_mean(
    tf.nn.sparse_softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  #optimizer = tf.train.AdagradOptimizer(0.01).minimize(loss)
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(0.05, global_step, 10000, 0.95)
  optimizer = tf.train.AdagradOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(model(tf_train_dataset, 1.0, shape))
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset, 1.0, shape))
  test_prediction = tf.nn.softmax(model(tf_test_dataset, 1.0, shape))
    
  saver = tf.train.Saver()

num_steps = 100001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size)]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0): 
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
  save_path = saver.save(session, "CNN_1.ckpt")
  print("Model saved in file: %s" % save_path)

Initialized
Minibatch loss at step 0: 2.453111
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 500: 2.262278
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 1000: 2.193046
Minibatch accuracy: 25.0%
Validation accuracy: 10.0%
Minibatch loss at step 1500: 2.287955
Minibatch accuracy: 15.6%
Validation accuracy: 10.0%
Minibatch loss at step 2000: 1.888970
Minibatch accuracy: 29.7%
Validation accuracy: 19.6%
Minibatch loss at step 2500: 0.379708
Minibatch accuracy: 92.2%
Validation accuracy: 82.8%
Minibatch loss at step 3000: 0.377859
Minibatch accuracy: 89.1%
Validation accuracy: 85.2%
Minibatch loss at step 3500: 0.319197
Minibatch accuracy: 90.6%
Validation accuracy: 87.0%
Minibatch loss at step 4000: 0.339692
Minibatch accuracy: 92.2%
Validation accuracy: 87.5%
Minibatch loss at step 4500: 0.216875
Minibatch accuracy: 92.2%
Validation accuracy: 87.9%
Minibatch loss at step 5000: 0.217608
Minibatch accuracy: 92.2%
Validation accurac

In [49]:
batch_size = 64
patch_size1 = 5
patch_size2 = 3
patch_size3 = 1
depth1 = 16
depth2 = 32
depth3 = 96
num_hidden1 = 64
num_hidden2 = 32
shape=[batch_size, image_size, image_size, num_channels]

# Construct a Inception Module CNN.
# C1: convolutional layer, batch_size x 28 x 28 x 16, convolution size: 5 x 5 x 1 x 16
# P2: max pool layer, batch_size x 14 x 14 x 16
# C3: convolutional layer, batch_size x 10 x 10 x 32, convolution size: 5 x 5 x 16 x 32

# Inception Module:
# Inception & Concat: output batch_size x 5 x 5 x 96
# C6: convolutional layer, batch_size x 5 x 5 x 32, convolution size: 1 x 1 x 96 x 32
# C7: convolutional layer, batch_size x 1 x 1 x 64, convolution size: 5 x 5 x 32 x 64
# Dropout layer
# F8: fully-connected layer, weight size: 64 x 16
# Output layer, weight size: 16 x 10

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.int64, shape=(batch_size))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.get_variable("W1", shape=[patch_size1, patch_size1, num_channels, depth1],\
           initializer=tf.contrib.layers.xavier_initializer_conv2d())
  layer1_biases = tf.Variable(tf.constant(1.0, shape=[depth1]), name='B1')

  layer2_weights = tf.get_variable("W2", shape=[patch_size1, patch_size1, depth1, depth2],\
           initializer=tf.contrib.layers.xavier_initializer_conv2d())
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth2]), name='B2')

  layer6_weights = tf.get_variable("W6", shape=[patch_size3, patch_size3, depth3, depth2],\
           initializer=tf.contrib.layers.xavier_initializer())
  layer6_biases = tf.Variable(tf.constant(1.0, shape=[depth2]), name='B6')

  layer3_weights = tf.get_variable("W3", shape=[patch_size1, patch_size1, depth2, num_hidden1],\
           initializer=tf.contrib.layers.xavier_initializer())
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden1]), name='B3')

  layer4_weights = tf.get_variable("W4", shape=[num_hidden1, num_hidden2],\
           initializer=tf.contrib.layers.xavier_initializer())
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden2]), name='B4')

  layer5_weights = tf.get_variable("W5", shape=[num_hidden2, num_labels],\
           initializer=tf.contrib.layers.xavier_initializer())
  layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='B5')
    
  # Inception Variables.
  incep_11_w = tf.get_variable("I11W", shape=[1, 1, depth2, 16],\
           initializer=tf.contrib.layers.xavier_initializer_conv2d())
  incep_11_b = tf.Variable(tf.constant(1.0, shape=[16]), name='I11B')

  incep_12_w = tf.get_variable("I12W", shape=[patch_size1, patch_size1, 16, 32],\
           initializer=tf.contrib.layers.xavier_initializer_conv2d())
  incep_12_b = tf.Variable(tf.constant(1.0, shape=[32]), name='I12B')

  incep_21_w = tf.get_variable("I21W", shape=[1, 1, depth2, 16],\
           initializer=tf.contrib.layers.xavier_initializer_conv2d())
  incep_21_b = tf.Variable(tf.constant(1.0, shape=[16]), name='I21B')

  incep_22_w = tf.get_variable("I22W", shape=[patch_size2, patch_size2, 16, 32],\
           initializer=tf.contrib.layers.xavier_initializer_conv2d())
  incep_22_b = tf.Variable(tf.constant(1.0, shape=[32]), name='I22B')

  incep_31_w = tf.get_variable("I31W", shape=[1, 1, depth2, 16],\
           initializer=tf.contrib.layers.xavier_initializer_conv2d())
  incep_31_b = tf.Variable(tf.constant(1.0, shape=[16]), name='I31B')

  incep_41_w = tf.get_variable("I41W", shape=[1, 1, depth2, 16],\
           initializer=tf.contrib.layers.xavier_initializer_conv2d())
  incep_41_b = tf.Variable(tf.constant(1.0, shape=[16]), name='I41B')
    
  # Model.
  def model(data, keep_prob, shape):
    LCN = LecunLCN(data, shape)
    conv = tf.nn.conv2d(LCN, layer1_weights, [1,1,1,1], 'VALID', name='C1')
    hidden = tf.nn.relu(conv + layer1_biases)
    lrn = tf.nn.local_response_normalization(hidden)
    sub = tf.nn.max_pool(lrn, [1,2,2,1], [1,2,2,1], 'SAME', name='P2')
    conv = tf.nn.conv2d(sub, layer2_weights, [1,1,1,1], padding='VALID', name='C3')
    hidden = tf.nn.relu(conv + layer2_biases)
    lrn = tf.nn.local_response_normalization(hidden)
    
    # Inception Module
    hidden = Inception(lrn)

    conv = tf.nn.conv2d(hidden, layer6_weights, [1,1,1,1], padding='SAME', name='C6')
    hidden = tf.nn.relu(conv + layer6_biases)
    lrn = tf.nn.local_response_normalization(hidden)
    conv = tf.nn.conv2d(lrn, layer3_weights, [1,1,1,1], padding='VALID', name='C7')
    hidden = tf.nn.relu(conv + layer3_biases)
    lrn = tf.nn.local_response_normalization(hidden)
    hidden = tf.nn.dropout(lrn, keep_prob)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])    
    hidden = tf.nn.relu(tf.matmul(reshape, layer4_weights) + layer4_biases)
    return tf.matmul(hidden, layer5_weights) + layer5_biases

  def Inception(input_tensor):
        # Branch 1
        conv1 = tf.nn.conv2d(input_tensor, incep_11_w, [1,1,1,1], padding='SAME', name='I1C1')
        hidden1 = tf.nn.relu(conv1 + incep_11_b)
        lrn1 = tf.nn.local_response_normalization(hidden1)
        conv1 = tf.nn.conv2d(lrn1, incep_12_w, [1,2,2,1], padding='SAME', name='I1C2')
        hidden1 = tf.nn.relu(conv1 + incep_12_b)
        lrn1 = tf.nn.local_response_normalization(hidden1)
        # Branch 2
        conv2 = tf.nn.conv2d(input_tensor, incep_21_w, [1,1,1,1], padding='SAME', name='I2C1')
        hidden2 = tf.nn.relu(conv2 + incep_21_b)
        lrn2 = tf.nn.local_response_normalization(hidden2)
        conv2 = tf.nn.conv2d(lrn2, incep_22_w, [1,2,2,1], padding='SAME', name='I2C2')
        hidden2 = tf.nn.relu(conv2 + incep_22_b)
        lrn2 = tf.nn.local_response_normalization(hidden2)
        # Branch 3
        sub3 = tf.nn.avg_pool(input_tensor, [1,3,3,1], [1,2,2,1], 'SAME', name='I3P1')
        conv3 = tf.nn.conv2d(sub3, incep_31_w, [1,1,1,1], padding='SAME', name='I3C2')
        hidden3 = tf.nn.relu(conv3 + incep_31_b)
        lrn3 = tf.nn.local_response_normalization(hidden3)
        # Branch 4
        conv4 = tf.nn.conv2d(input_tensor, incep_41_w, [1,2,2,1], padding='SAME', name='I4C1')
        hidden4 = tf.nn.relu(conv4 + incep_41_b)
        lrn4 = tf.nn.local_response_normalization(hidden4)
        # Concat
        output_tensor = tf.concat(3, [lrn1, lrn2, lrn3, lrn4], name='CONCAT')
        return output_tensor
  
  # Training computation.
  logits = model(tf_train_dataset, 1, shape)
  loss = tf.reduce_mean(
    tf.nn.sparse_softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  #optimizer = tf.train.AdagradOptimizer(0.01).minimize(loss)
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(0.01, global_step, 10000, 0.95)
  optimizer = tf.train.AdagradOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(model(tf_train_dataset, 1.0, shape))
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset, 1.0, shape))
  test_prediction = tf.nn.softmax(model(tf_test_dataset, 1.0, shape))
    
  saver = tf.train.Saver()

num_steps = 100001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()  
  reader = tf.train.NewCheckpointReader("CNN_1.ckpt")
  reader.get_variable_to_shape_map()
  #saver.restore(session, "CNN_1.ckpt")
  print("Model restored.")  
  
  
  #tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size)]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    #print(predictions[:])
    if (step % 500 == 0): 
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
  save_path = saver.save(session, "CNN_2.ckpt")
  print("Model saved in file: %s" % save_path)

Model restored.
Initialized
Minibatch loss at step 0: 3.212271
Minibatch accuracy: 9.4%
Validation accuracy: 10.0%
Minibatch loss at step 500: 2.268531
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 1000: 2.187550
Minibatch accuracy: 25.0%
Validation accuracy: 10.0%
Minibatch loss at step 1500: 2.286384
Minibatch accuracy: 15.6%
Validation accuracy: 10.0%
Minibatch loss at step 2000: 2.227596
Minibatch accuracy: 15.6%
Validation accuracy: 10.0%
Minibatch loss at step 2500: 2.332437
Minibatch accuracy: 9.4%
Validation accuracy: 10.0%
Minibatch loss at step 3000: 2.257286
Minibatch accuracy: 17.2%
Validation accuracy: 10.0%


KeyboardInterrupt: 